In [1]:
# !pip install transformers
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import pandas as pd
import glob
from read_csv_gz import read_csv_gz
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
import sys
print(sys.executable)


/opt/miniconda3/envs/erdos_spring_2025/bin/python


In [3]:
ed_diagnosis_df = read_csv_gz("ed_data/diagnosis.csv.gz")
ed_edstays_df = read_csv_gz("ed_data/edstays.csv.gz")
ed_medrecon_df = read_csv_gz("ed_data/medrecon.csv.gz")
ed_pyxis_df = read_csv_gz("ed_data/pyxis.csv.gz")
ed_triage_df = read_csv_gz("ed_data/triage.csv.gz")
ed_vitalsign_df = read_csv_gz("ed_data/vitalsign.csv.gz")

In [4]:
def text_gen(x):
    return f"You are a medical expert assisting in an Emergency Department (ED). Your task is to assess the **diagnostic complexity** of the following chief complaint on a scale of 1-10:\n- **1-3 (Low Complexity):** Easily diagnosable, requires minimal tests.\n- **4-6 (Moderate Complexity):** Needs some testing, single specialty.\n- **7-10 (High Complexity):** Requires multiple tests, possible admission, specialist consults.\nChief Complaint: **{x}**\nPlease only return an integer of Complexity Score (1-10):"

In [5]:
print(ed_triage_df.shape[0])

425087


In [6]:
def converter(df, m):
    ans = []
    temp = []
    for i in range(df.shape[0]):
        text = text_gen(df.iloc[i])
        temp.append(text)
        if (i + 1) % m == 0:
            if len(temp) != 0 :
                ans.append(temp)
            temp = []
    if len(temp) != 0:
        ans.append(temp)
    return ans

In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [8]:
def indefence_prompt(batch_lst, tokenizer, model, device="cuda" if torch.cuda.is_available() else "cpu"):
    ans = []
    for batch in tqdm(batch_lst):
        bs = len(batch)
        inputs = tokenizer(batch, padding=True, padding_side='left', return_tensors="pt").to(device)
        output_sequences = model.generate(**inputs, max_length=200)
        for i in range(bs):
            generated_text = tokenizer.decode(output_sequences[i, :], skip_special_tokens=True)
            ans.append(generated_text)
    return pd.Series(ans)
        
        


In [9]:
lst_prompt = converter(ed_triage_df["chiefcomplaint"].head(234), 10)

In [10]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())  # Should print True if CUDA is installed correctly


2.6.0
False


In [11]:
model_name = "bigscience/bloom-560m"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float32)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

In [13]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

BloomForCausalLM(
  (transformer): BloomModel(
    (word_embeddings): Embedding(250880, 1024)
    (word_embeddings_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (h): ModuleList(
      (0-23): 24 x BloomBlock(
        (input_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (self_attention): BloomAttention(
          (query_key_value): Linear(in_features=1024, out_features=3072, bias=True)
          (dense): Linear(in_features=1024, out_features=1024, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (post_attention_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): BloomMLP(
          (dense_h_to_4h): Linear(in_features=1024, out_features=4096, bias=True)
          (gelu_impl): BloomGelu()
          (dense_4h_to_h): Linear(in_features=4096, out_features=1024, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (

In [14]:
res = indefence_prompt(lst_prompt, tokenizer, model, device="cuda" if torch.cuda.is_available() else "cpu")

100%|██████████| 24/24 [08:48<00:00, 22.00s/it]


In [18]:
lst_prompt = ['You are a student at UCSD and you took a horrible course Math 20D, ']
res = indefence_prompt(lst_prompt, tokenizer, model, device="cuda" if torch.cuda.is_available() else "cpu")

  0%|          | 0/1 [00:32<?, ?it/s]


IndexError: index 1 is out of bounds for dimension 0 with size 1

In [20]:
batch = text_gen('hypotension')
inputs = tokenizer(batch, padding=True, padding_side='left', return_tensors="pt").to(device)
output_sequences = model.generate(**inputs, max_length=200)
generated_text = tokenizer.decode(output_sequences[0, :], skip_special_tokens=True)
print(generated_text)

You are a medical expert assisting in an Emergency Department (ED). Your task is to assess the **diagnostic complexity** of the following chief complaint on a scale of 1-10:
- **1-3 (Low Complexity):** Easily diagnosable, requires minimal tests.
- **4-6 (Moderate Complexity):** Needs some testing, single specialty.
- **7-10 (High Complexity):** Requires multiple tests, possible admission, specialist consults.
Chief Complaint: **hypotension**
Please only return an integer of Complexity Score (1-10):
1. Low Complexity
2. Moderate Complexity
3. High Complexity
4. Needs some testing
5. Specialty consults
6. Emergency Department (ED) admission
7. Emergency Department (ED) admission
8. Emergency Department (ED) admission
9. Emergency Department (ED) admission
10. Emergency Department (


In [15]:
print(res)

0      You are a medical expert assisting in an Emerg...
1      You are a medical expert assisting in an Emerg...
2      You are a medical expert assisting in an Emerg...
3      You are a medical expert assisting in an Emerg...
4      You are a medical expert assisting in an Emerg...
                             ...                        
229    You are a medical expert assisting in an Emerg...
230    You are a medical expert assisting in an Emerg...
231    You are a medical expert assisting in an Emerg...
232    You are a medical expert assisting in an Emerg...
233    You are a medical expert assisting in an Emerg...
Length: 234, dtype: object


In [17]:
print(res[1])

You are a medical expert assisting in an Emergency Department (ED). Your task is to assess the **diagnostic complexity** of the following chief complaint on a scale of 1-10:
- **1-3 (Low Complexity):** Easily diagnosable, requires minimal tests.
- **4-6 (Moderate Complexity):** Needs some testing, single specialty.
- **7-10 (High Complexity):** Requires multiple tests, possible admission, specialist consults.
Chief Complaint: **Abd pain, Abdominal distention**
Please only return an integer of Complexity Score (1-10):
1. Low Complexity
2. Moderate Complexity
3. High Complexity
4. Needs some testing
5. Specialty consults
6. Admission
7. Specialty consults
8. Admission
9. Specialty consults
10. Admission
Please note that
